<a href="https://colab.research.google.com/github/ArtIncor/Embedding-2025-2026/blob/main/Embeddings_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Введение**

На прошлом занятии мы научились получать «семантические портреты» белков с помощью языковых моделей (ESM2). Мы увидели, что эти портреты, или эмбеддинги, несут в себе информацию о структуре и, возможно, функции белка.

**Цель этого задания** — копнуть глубже. Вы выступите в роли биоинформатика-исследователя. Ваша задача — взять один белок, изучить его со всех сторон, используя его 3D-структуру, и сопоставить ваши находки с тем, что «видит» языковая модель.

**Ключевые инструменты:**
*   **BioPython:** для парсинга PDB-файлов и работы со структурой.
*   **NumPy/Pandas:** для вычислений и обработки данных.
*   **Matplotlib/Seaborn:** для построения красивых графиков.
*   **PyMOL:** для 3D-визуализации (вы будете делать скриншоты или генерировать скрипты).
*   **Transformers (HuggingFace):** для получения эмбеддингов из ESM.

## **Задание: создайте паспорт белка**

Вам предстоит провести комплексный анализ одного белка из предложенного списка. Результатом вашей работы станет этот же ноутбук, но заполненный вашим кодом, графиками и, что самое важное, **вашими выводами**.

**Выберите один белок из списка:**
*   Цитохром b6f с бета-каротином
*   Сывороточный альбумин человека с жирной кислотой
*   Рецептор ретиноевой кислоты с лигандом
*   Сывороточный альбумин человека, апо-форма, без специфического лиганда - интересный случай для сравнения
*   Оранжевый каротиноидный белок (OCP)
*   Миоглобин с гемом
*   Если хотите - любой другой белок :)

**Ваш "паспорт" должен включать следующие разделы:**

1. Структурный анализ:

    *   Загрузите PDB-структуру. Определите, какие цепи являются белком, а какой остаток — лигандом.
    *   Найдите все аминокислоты, находящиеся в контакте с лигандом (например, на расстоянии < 4.5 Ангстрем). Это и будет наш "карман".
    *   **Визуализируйте результат в PyMOL.** Сделайте красивую картинку, где белок показан в виде ленты (cartoon), лиганд — в виде шариков (spheres), а аминокислоты кармана — в виде палочек (sticks). Подпишите несколько ключевых остатков. Вставьте скриншот в ноутбук.

2. Последовательностный анализ:

    *   Извлеките аминокислотную последовательность белка.
    *   **"Подсветите" карман:** выведите последовательность так, чтобы аминокислоты, входящие в сайт связывания, были написаны заглавными буквами, а все остальные — строчными.
    *   **Сравните состав:** посчитайте аминокислотный состав кармана и сравните его с составом всего белка. Постройте гистограмму, показывающую, какие аминокислоты обогащают карман. Сделайте вывод (например, "карман оказался более гидрофобным").

3. Семантический анализ:

    *   Получите эмбеддинги для каждого остатка вашего белка с помощью любой белковой языковой модели. Можете взять ту, что мы использовали в прошлом ноутбуке (ESM2), а можете сами поискать другие варианты или сравнить несколько разных - творчество здесь не только приветствуется, но и является ключевым условием для достижения наших целей.
    *   Разделите все эмбеддинги на две группы: "карман" и "не карман", используя разметку из прошлых шагов.
    *   Используя методы снижения размерности (еще раз посмотрите, какие они бывают и какой вам лучше всего подойдет), постройте на двумерном scatter plot'е все аминокислоты вашего белка. Раскрасьте их в зависимости от принадлежности к карману (двумя цветами - да/нет).
    *   **Постройте ящик с усами (box plot).** Это ключевой график. Он должен сравнивать распределение какого-то параметра для двух групп. Например, можно взять главную компоненту эмбеддинга после снижения размерности или косинусное сходство каждого эмбеддинга с усредненным эмбеддингом кармана (ожидается, что эмбеддинги кармана будут около нуля, а не-кармана - отличны от нуля). Ваша задача - найти такую метрику, по которой будет видно наиболее достоверное различие.
    *   **Сравните эти две группы.** Достаточно ли информации в эмбеддингах, чтобы отличить остаток из кармана от случайного остатка на поверхности? ЧТо вы видите на scatter plot'ах? А на ящиках с усами?
    *   Сделайте вывод, основываясь на графике.

**Формат сдачи:** Заполненный ноутбук. Каждая секция должна содержать ячейки с кодом, результат их выполнения (графики, таблицы, текст) и ячейку с вашими выводами. Удачи!


In [7]:
# --- ЧАСТЬ 0: ПОДГОТОВКА ---

# Установка необходимых библиотек
!pip install biopython transformers torch

# Импорты
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Bio.PDB import PDBParser, Selection, PDBIO # Та самая Biopython про которую я говорил. Ваша задача - разобраться с этой библиотекой в первую очередь
from Bio.SeqUtils import seq1
from transformers import AutoTokenizer, EsmModel
from Bio.PDB import PDBParser, Selection, PDBIO, NeighborSearch

# Настройки для графиков
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 7)

print("Все библиотеки готовы к работе!")

Все библиотеки готовы к работе!


In [4]:
# --- ЧАСТЬ 1: СТРУКТУРНЫЙ АНАЛИЗ (НАЧАЛО) ---

# Выберите ваш PDB ID и ID лиганда
PDB_ID = "3MG1"
LIGAND_ID = "ECH" # Beta-carotene

# Загружаем структуру из PDB
!wget -q https://files.rcsb.org/download/{PDB_ID}.pdb

# Создаем парсер и загружаем структуру
parser = PDBParser(QUIET=True)
structure = parser.get_structure(PDB_ID, f"{PDB_ID}.pdb")
model = structure[0] # Берем первую модель из PDB

# --- ВАШ КОД ЗДЕСЬ ---
ligand = None
# Перебираем все остатки в структуре
for chain in model:
    for residue in chain:
        # Проверяем по названию остатка
        if residue.get_resname() == LIGAND_ID:
            ligand = residue
            break
    if ligand:
        break

if ligand:
    ligand_atoms = list(ligand.get_atoms())
    print(f"Лиганд {LIGAND_ID} найден в цепи {ligand.get_parent().id}.")
else:
    print(f"Ошибка: Лиганд {LIGAND_ID} не найден.")

Лиганд ECH найден в цепи A.


In [5]:
# 2. Найдите все атомы белка.
#    Подсказка: можно использовать Selection.unfold_entities(model, 'A') для получения списка атомов
protein_atoms = []
# ... ваш код ...
protein_atoms = Selection.unfold_entities(model, 'A') # Получаем список всех атомов
print(f"Найдено {len(protein_atoms)} атомов белка.")

Найдено 5497 атомов белка.


In [8]:
# 3. Напишите функцию, которая находит остатки кармана
def find_pocket_residues(protein_atoms, ligand, cutoff=4.5):
    """
    Находит все аминокислотные остатки, у которых есть хотя бы один атом
    на расстоянии <= cutoff от любого атома лиганда.
    Возвращает список объектов Residue.
    """
    pocket_residues = set() # Используем set, чтобы избежать дубликатов (это как массив, но элементы в нем не могут повторяться)

    if not ligand_atoms:
        return []

    # 1. Извлекаем все атомы лиганда
    ligand_atom_list = ligand_atoms

    # 2. Создаем структуру для поиска соседей (ускоряет процесс)
    # Используем NeighborSearch для быстрого нахождения атомов белка рядом с лигандом
    # Сначала извлечем координаты всех атомов лиганда для создания объекта.

    # 3. Находим все атомы белка, близкие к лиганду
    ns = NeighborSearch(protein_atoms)

    # Ищем атомы белка в пределах cutoff от любого атома лиганда
    # Эта функция делает за нас вложенный цикл:
    # Ищет ближайшие атомы белка для каждого атома лиганда.
    close_protein_atoms = set()
    for l_atom in ligand_atom_list:
        neighbors = ns.search(l_atom.get_coord(), cutoff, level='A') # level='A' - возвращает список атомов
        close_protein_atoms.update(neighbors)

    # 4. Извлекаем уникальные родительские остатки (Residue)
    pocket_residues = set()
    for atom in close_protein_atoms:
        pocket_residues.add(atom.get_parent())

    return list(pocket_residues)

# Получаем остатки кармана
pocket = find_pocket_residues(protein_atoms, ligand)
print(f"Найдено {len(pocket)} остатков в кармане.")

# 4. Сделайте скриншот из PyMOL и вставьте его в текстовую ячейку ниже.

Найдено 40 остатков в кармане.
